# Exercise 1: Wiener Filter

In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### (a) Optimal filter weights

In [ ]:
R_x = np.array([[5, -1, -2],[-1, 5, -1],[-2, -1, 5]])
r_yx = np.array([1, 5.3, -3.9]).reshape((3,1))
R_x_inv = np.linalg.inv(R_x)

w_o = np.dot(R_x_inv,  r_yx)
print("Optimal weights are:\n", w_o)
print("")
print("inverse of R_x is:\n", R_x_inv)


# Exercise 2: Steepest Gradient Descent

### (b) Calculate eigenvalues of R_x

In [ ]:
eigenvalues = np.linalg.eigvals(R_x)
print("eigenvalues of R_x are:\n", eigenvalues)
# alpha between 0 and 1/7

### (c) Implementing the filter update rule of the GD algorithm 

In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv('assignment1_data.csv', header=None)

# Assign column names assuming the first column is x and the second column is y
df.columns = ['x', 'y']

# Print the DataFrame to verify the data
print(df)

In [ ]:
weights = np.array([0,0,0]).reshape(3,1)
J, weights_history,x_array, y_array = [], [], [], []
alpha = 1e-3
R_x = np.array(([5, -1, -2],[-1, 5, -1],[-2, -1, 5]))
r_yx = np.array([1, 5.3, -3.9]).reshape((3,1))

for i in range(2,len(df)):

    #Predict
    x = np.array([df['x'][i-2],df['x'][i-1],df['x'][i]]).reshape(1,3)[0] #input
    x_array.append(x)
    y_hat = np.dot(x, weights)[0] #Prediction

    #Error
    y = df['y'][i] #ground truth y
    y_array.append(y)
    J.append(((y-y_hat)**2)) #calculate MSE loss

    #Update
    weights_history.append(weights) #Add current weights to history
    weights = weights + 2*alpha*(r_yx - np.dot(R_x, weights)) #Update weights according to known statistics

print("Final weights are:\n", weights)

In [ ]:
len(x_array)

In [ ]:
w0,w1,w2 = [],[],[]
for i in range(len(weights_history)):
    w0.append(weights_history[i][0])
    w1.append(weights_history[i][1])
    w2.append(weights_history[i][2])


In [18]:
w0 = np.linspace(-2.5,2.5)
w1 = np.linspace(-2.5,2.5)
w2 = np.ones(100)*-0.5

# eerst J uitrekenen voor meshgrid
for i, w0 in enumerate(w0):
    for j, w1 in enumerate(w1):

        #J[i,j] = Error



W0, W1 = np.meshgrid(w0, w1)
plt.contour(W0, W1, J)

50

# Exercise 3: Newton’s Method

In [ ]:
weights = np.array([0,0,0]).reshape(3,1)
J, weights_history = [], []
alpha = 1e-3
R_x = np.array(([5, -1, -2],[-1, 5, -1],[-2, -1, 5]))
r_yx = np.array([1, 5.3, -3.9]).reshape((3,1))

for i in range(2,len(df)):

    #Predict
    x = np.array([df['x'][i-2],df['x'][i-1],df['x'][i]]).reshape(1,3)[0] #input
    y_hat = np.dot(x, weights)[0] #Prediction

    #Error
    y = df['y'][i] #ground truth y
    J.append(((y-y_hat)**2)) #calculate MSE loss

    #Update
    weights_history.append(weights) #Add current weights to history
    weights = weights + np.dot(2*alpha*np.linalg.inv(R_x),(r_yx - np.dot(R_x, weights))) #Update weights according to the newton rule
print("Final weights are:\n", weights)

# Exercise 4: LMS and NLMS

In [ ]:
weights = np.array([0,0,0]).reshape(3,1)
J, weights_history = [], []
alpha = 1e-3
R_x = np.array(([5, -1, -2],[-1, 5, -1],[-2, -1, 5]))
r_yx = np.array([1, 5.3, -3.9]).reshape((3,1))

for i in range(2,len(df)):

    #Predict
    x = np.array([df['x'][i-2],df['x'][i-1],df['x'][i]]).reshape(1,3)[0] #input
    y_hat = np.dot(x, weights)[0] #Prediction

    #Error
    y = df['y'][i] #ground truth y
    J.append(y-y_hat) #calculate ls loss

    #Update
    weights_history.append(weights) #Add current weights to history
    weights = weights + np.dot(2*alpha*np.linalg.inv(R_x),(r_yx - np.dot(R_x, weights))) #Update weights according to the newton rule
print("Final weights are:\n", weights)

# Exercise 5: RLS

# Exercise 6

# Exercise 7